# Dense Neural Network (NN) Exploration

In this notebook, we will lay out our approach for a Neural Network approach to predict a pose based on landmark. Additionally, we are storing out angle calculation and perspectively our angle error calculation.

In [1]:
import numpy as np
import pandas as pd

## Neural Network

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers

In [ ]:
# X = data.drop(columns = 'label')
# y = 

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size = 0.3)

In [ ]:
# def initialize_model():
    
#     #############################
#     #  1 - Model architecture   #
#     ############################# 
    
#     model = models.Sequential()
#     model.add(layers.Dense(20, activation='relu', input_dim=51))
#     model.add(layers.Dense(10, activation='relu', input_dim=51))
#     model.add(layers.Dense(10, activation='relu', input_dim=51))
#     model.add(layers.Dense(5, activation='softmax'))
#     # $CHALLENGIFY_END
    
#     #############################
#     #  2 - Optimization Method  #
#     #############################
#     model.compile(loss='categorical_crossentropy', # different from binary_crossentropy because we have multiple classes
#                   optimizer='adam', 
#                   metrics=['accuracy']) 

#     return model 


# model = initialize_model()

In [ ]:
# model.summary()

In [ ]:
# model = initialize_model()

# history = model.fit(X_train, y_train, 
#                     epochs=50, 
#                     batch_size=16, 
#                     verbose=0)

In [ ]:
# def plot_history(history):
#     plt.plot(history.history['loss'])
#     plt.title('Model loss')
#     plt.ylabel('Loss')
#     plt.xlabel('Epoch')
#     plt.show()

# plot_history(history)

In [ ]:
# results = model.evaluate(X_test, y_test, verbose=0)
# print(results[1])

## Masaki

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed
import numpy as np
import cv2
import os
import pandas as pd

# Import matplotlib libraries
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.patches as patches

# Some modules to display an animation using imageio.
import imageio
from IPython.display import HTML, display


In [4]:
# Model Initialization
model_name = "movenet_lightning"
module = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
input_size = 192
model = module.signatures['serving_default']

In [5]:
# Choose image file and preprocess

image_path = "../clean/00000320.jpg"
image = tf.io.read_file(image_path)
image = tf.image.decode_jpeg(image)

# Resize and pad the image to keep the aspect ratio and fit the expected size.
input_image = tf.expand_dims(image, axis=0)
input_image = tf.image.resize_with_pad(input_image, input_size, input_size)


In [6]:
# Detect landmarks on image
model = module.signatures["serving_default"]
input_image = tf.cast(input_image, dtype=tf.int32)
input_image = input_image[..., :3]
outputs = model(input_image)

# 1D array of xyz coordinates. X1, Y1, Z1, X2, ... , Yn, Zn format
xyz = outputs["output_0"].numpy().reshape(51).tolist()


2023-11-21 16:30:42.425583: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [7]:
# Helper method for drawing landmarks
def draw_prediction_on_image(
    image, keypoints_with_scores, crop_region=None, close_figure=False,
    output_image_height=None):
  """Draws the keypoint predictions on image.

  Args:
    image: A numpy array with shape [height, width, channel] representing the
      pixel values of the input image.
    keypoints_with_scores: A numpy array with shape [1, 1, 17, 3] representing
      the keypoint coordinates and scores returned from the MoveNet model.
    crop_region: A dictionary that defines the coordinates of the bounding box
      of the crop region in normalized coordinates (see the init_crop_region
      function below for more detail). If provided, this function will also
      draw the bounding box on the image.
    output_image_height: An integer indicating the height of the output image.
      Note that the image aspect ratio will be the same as the input image.

  Returns:
    A numpy array with shape [out_height, out_width, channel] representing the
    image overlaid with keypoint predictions.
  """
  height, width, channel = image.shape
  aspect_ratio = float(width) / height
  fig, ax = plt.subplots(figsize=(12 * aspect_ratio, 12))
  # To remove the huge white borders
  fig.tight_layout(pad=0)
  ax.margins(0)
  ax.set_yticklabels([])
  ax.set_xticklabels([])
  plt.axis('off')

  im = ax.imshow(image)
  line_segments = LineCollection([], linewidths=(4), linestyle='solid')
  ax.add_collection(line_segments)
  # Turn off tick labels
  scat = ax.scatter([], [], s=60, color='#FF1493', zorder=3)

  (keypoint_locs, keypoint_edges,
   edge_colors) = _keypoints_and_edges_for_display(
       keypoints_with_scores, height, width)

  line_segments.set_segments(keypoint_edges)
  line_segments.set_color(edge_colors)
  if keypoint_edges.shape[0]:
    line_segments.set_segments(keypoint_edges)
    line_segments.set_color(edge_colors)
  if keypoint_locs.shape[0]:
    scat.set_offsets(keypoint_locs)

  if crop_region is not None:
    xmin = max(crop_region['x_min'] * width, 0.0)
    ymin = max(crop_region['y_min'] * height, 0.0)
    rec_width = min(crop_region['x_max'], 0.99) * width - xmin
    rec_height = min(crop_region['y_max'], 0.99) * height - ymin
    rect = patches.Rectangle(
        (xmin,ymin),rec_width,rec_height,
        linewidth=1,edgecolor='b',facecolor='none')
    ax.add_patch(rect)

  fig.canvas.draw()
  image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
  image_from_plot = image_from_plot.reshape(
      fig.canvas.get_width_height()[::-1] + (3,))
  plt.close(fig)
  if output_image_height is not None:
    output_image_width = int(output_image_height / height * width)
    image_from_plot = cv2.resize(
        image_from_plot, dsize=(output_image_width, output_image_height),
         interpolation=cv2.INTER_CUBIC)
  return image_from_plot

In [8]:
# Get list of image file names in given directory
dir_path_train_dd = "../clean"
file_names = []
for entry in os.listdir(dir_path_train_dd):
    file_names.append(entry)

In [9]:
# Create DF of xyz values
df = []

for file in file_names:
    dir_path_train_dd = f"../clean/{file}"
    if not file.endswith((".jpg", ".png")):
        pass
    else:
        print(dir_path_train_dd)
        image = tf.io.read_file(dir_path_train_dd)
        image = tf.image.decode_jpeg(image)
        input_image = tf.expand_dims(image, axis=0)
        input_image = tf.image.resize_with_pad(input_image, input_size, input_size)
        input_image = input_image[..., :3]

        model = module.signatures["serving_default"]
        input_image = tf.cast(input_image, dtype=tf.int32)
        outputs = model(input_image)
        df.append(outputs["output_0"].numpy().reshape(51).tolist())


../clean/00000372.jpg
../clean/00000414.jpg
../clean/00000158.jpg
../clean/00000164.jpg
../clean/00000170.jpg
../clean/00000366.png
../clean/00000206.jpg
../clean/00000213.jpg
../clean/00000165.jpg
../clean/00000159.jpg
../clean/00000198.png
../clean/00000417.jpg
../clean/00000403.jpg
../clean/00000173.jpg
../clean/00000239.jpg
../clean/00000210.jpg
../clean/00000204.jpg
../clean/00000238.jpg
../clean/00000199.jpg
../clean/00000166.jpg
../clean/00000172.jpg
../clean/00000416.jpg
../clean/00000358.jpg
../clean/00000412.jpg
../clean/00000406.jpg
../clean/00000360.jpg
../clean/00000176.jpg
../clean/00000162.jpg
../clean/00000189.jpg
../clean/00000228.png
../clean/00000200.jpg
../clean/00000214.jpg
../clean/00000229.jpg
../clean/00000201.jpg
../clean/00000349.png
../clean/00000163.jpg
../clean/00000375.jpg
../clean/00000377.jpg
../clean/00000161.jpg
../clean/00000175.jpg
../clean/00000388.png
../clean/00000217.jpg
../clean/00000216.jpg
../clean/00000160.jpg
../clean/00000376.jpg
../clean/0

In [10]:
data = pd.DataFrame(df)

In [11]:
data["label"] = "downdog"

In [12]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,label
0,0.614686,0.366069,0.722570,0.612122,0.350986,0.643456,0.612995,0.350938,0.715975,0.563880,...,0.515043,0.705209,0.887617,0.685450,0.883067,0.797601,0.667293,0.864593,0.652384,downdog
1,0.534696,0.722514,0.564541,0.529799,0.733964,0.405342,0.526650,0.732507,0.399033,0.498358,...,0.471535,0.375810,0.738871,0.586280,0.311128,0.580621,0.628086,0.273356,0.892581,downdog
2,0.569520,0.529188,0.600391,0.572470,0.513902,0.517100,0.575602,0.515343,0.508116,0.547930,...,0.502984,0.773414,0.708213,0.674067,0.870783,0.614912,0.665915,0.863403,0.627864,downdog
3,0.621766,0.494117,0.670643,0.623084,0.508379,0.587127,0.625575,0.502040,0.504647,0.590072,...,0.494093,0.241079,0.877534,0.604555,0.187687,0.531326,0.631976,0.162686,0.756716,downdog
4,0.615906,0.424954,0.598805,0.618176,0.415418,0.541572,0.616567,0.410814,0.479552,0.595441,...,0.533408,0.626742,0.754802,0.652928,0.721405,0.676844,0.650494,0.716107,0.753108,downdog


In [112]:
# Create DF of xyz values
df = []
xyz = []
for file in file_names:
    dir_path_train_dd = f"../clean/{file}"
    if not file.endswith((".jpg", ".png")):
        pass
    else:
        print(dir_path_train_dd)
        image = tf.io.read_file(dir_path_train_dd)
        image = tf.image.decode_jpeg(image)
        input_image = tf.expand_dims(image, axis=0)
        input_image = tf.image.resize_with_pad(input_image, input_size, input_size)
        input_image = input_image[..., :3]

        model = module.signatures["serving_default"]
        input_image = tf.cast(input_image, dtype=tf.int32)
        outputs = model(input_image)
        df.append(outputs["output_0"].numpy().reshape(51).tolist())
        xyz.append(outputs["output_0"].numpy()[0][0])

../clean/00000372.jpg
../clean/00000414.jpg
../clean/00000158.jpg
../clean/00000164.jpg
../clean/00000170.jpg
../clean/00000366.png
../clean/00000206.jpg
../clean/00000213.jpg
../clean/00000165.jpg
../clean/00000159.jpg
../clean/00000198.png
../clean/00000417.jpg
../clean/00000403.jpg
../clean/00000173.jpg
../clean/00000239.jpg
../clean/00000210.jpg
../clean/00000204.jpg
../clean/00000238.jpg
../clean/00000199.jpg
../clean/00000166.jpg
../clean/00000172.jpg
../clean/00000416.jpg
../clean/00000358.jpg
../clean/00000412.jpg
../clean/00000406.jpg
../clean/00000360.jpg
../clean/00000176.jpg
../clean/00000162.jpg
../clean/00000189.jpg
../clean/00000228.png
../clean/00000200.jpg
../clean/00000214.jpg
../clean/00000229.jpg
../clean/00000201.jpg
../clean/00000349.png
../clean/00000163.jpg
../clean/00000375.jpg
../clean/00000377.jpg
../clean/00000161.jpg
../clean/00000175.jpg
../clean/00000388.png
../clean/00000217.jpg
../clean/00000216.jpg
../clean/00000160.jpg
../clean/00000376.jpg
../clean/0

In [40]:
xyz[0]

array([[[[0.61468613, 0.36606914, 0.7225705 ],
         [0.61212206, 0.35098642, 0.6434561 ],
         [0.6129949 , 0.3509376 , 0.7159747 ],
         [0.56388015, 0.3286766 , 0.58742386],
         [0.5627878 , 0.3274661 , 0.5724267 ],
         [0.5053564 , 0.37220737, 0.75928754],
         [0.5080349 , 0.37157354, 0.7369339 ],
         [0.6347408 , 0.26611096, 0.7730443 ],
         [0.6133088 , 0.26804513, 0.77417135],
         [0.70369303, 0.15653855, 0.71411836],
         [0.675892  , 0.1728544 , 0.58133334],
         [0.26079118, 0.6013876 , 0.8471483 ],
         [0.26600903, 0.60989666, 0.8646514 ],
         [0.5165872 , 0.7039327 , 0.8456954 ],
         [0.51504314, 0.70520854, 0.8876169 ],
         [0.6854501 , 0.8830673 , 0.7976013 ],
         [0.667293  , 0.86459327, 0.6523842 ]]]], dtype=float32)

In [55]:
xyz[8]

array([[[[0.7569082 , 0.41272148, 0.47615528],
         [0.7564088 , 0.39499462, 0.47568285],
         [0.7530427 , 0.3971178 , 0.48554584],
         [0.7332966 , 0.38527426, 0.5260484 ],
         [0.72380275, 0.3867472 , 0.59014755],
         [0.69625944, 0.4171415 , 0.67764705],
         [0.6746234 , 0.40753394, 0.6139774 ],
         [0.76735127, 0.33971903, 0.5570904 ],
         [0.7304582 , 0.3622514 , 0.4074675 ],
         [0.7940702 , 0.2639392 , 0.39999905],
         [0.77152634, 0.28561682, 0.1538964 ],
         [0.5045524 , 0.5742487 , 0.79458165],
         [0.50276434, 0.57377166, 0.74451977],
         [0.6691898 , 0.6178473 , 0.75238764],
         [0.6635809 , 0.6115875 , 0.72027856],
         [0.75192225, 0.72325325, 0.6977503 ],
         [0.7469103 , 0.72464246, 0.626944  ]]]], dtype=float32)

## Angle Calculator

In [260]:
image_landmarks = xyz[6]

In [262]:
lm = image_landmarks
lm

array([[0.66960835, 0.35614344, 0.5095179 ],
       [0.67016554, 0.33723804, 0.43833122],
       [0.6718497 , 0.33988258, 0.55845886],
       [0.6360858 , 0.30929214, 0.44375876],
       [0.6374579 , 0.31444016, 0.5736966 ],
       [0.58631724, 0.3324954 , 0.7359366 ],
       [0.58074266, 0.331631  , 0.7099772 ],
       [0.70875835, 0.27349722, 0.5079271 ],
       [0.6967046 , 0.26761028, 0.6038063 ],
       [0.7763366 , 0.1773637 , 0.52029693],
       [0.75954664, 0.19102556, 0.5520658 ],
       [0.40240896, 0.53494716, 0.68205947],
       [0.40276787, 0.5312333 , 0.6648428 ],
       [0.56703186, 0.63760805, 0.793809  ],
       [0.57365   , 0.6361871 , 0.8454825 ],
       [0.7234893 , 0.7441256 , 0.71151537],
       [0.7093338 , 0.7298353 , 0.6462628 ]], dtype=float32)

In [136]:
KEYPOINT_DICT = {
    'nose': 0,
    'left_eye': 1,
    'right_eye': 2,
    'left_ear': 3,
    'right_ear': 4,
    'left_shoulder': 5,
    'right_shoulder': 6,
    'left_elbow': 7,
    'right_elbow': 8,
    'left_wrist': 9,
    'right_wrist': 10,
    'left_hip': 11,
    'right_hip': 12,
    'left_knee': 13,
    'right_knee': 14,
    'left_ankle': 15,
    'right_ankle': 16
}

In [137]:
# landmark_dict = {
#     'landmarks_left_elbow' : (lm[5], lm[7], lm[9]),
#     'landmarks_right_elbow' : (lm[6], lm[8], lm[10]),
#     'landmarks_left_shoulder' : (lm[7], lm[9], lm[11]),
#     'landmarks_right_shoulder' : (lm[8], lm[10], lm[12]),
#     'landmarks_hip_left' : (lm[5], lm[11], lm[13]),
#     'landmarks_hip_right' : (lm[6], lm[12], lm[14]),
#     'landmarks_left_knee' : (lm[11], lm[13], lm[15]),
#     'landmarks_right_knee' : (lm[12], lm[14], lm[16])
#         }

In [263]:
def angle_function(lm):
    
    landmark_dict = {
    'landmarks_left_elbow' : (lm[5], lm[7], lm[9]),
    'landmarks_right_elbow' : (lm[6], lm[8], lm[10]),
    'landmarks_left_shoulder' : (lm[7], lm[9], lm[11]),
    'landmarks_right_shoulder' : (lm[8], lm[10], lm[12]),
    'landmarks_hip_left' : (lm[5], lm[11], lm[13]),
    'landmarks_hip_right' : (lm[6], lm[12], lm[14]),
    'landmarks_left_knee' : (lm[11], lm[13], lm[15]),
    'landmarks_right_knee' : (lm[12], lm[14], lm[16])
        }
    
    def angle_calculator(landmarks):

        # Converting the points into numpy arrays
        p1 = np.array(landmarks[0])
        p2 = np.array(landmarks[1])
        p3 = np.array(landmarks[2])

        # Creating the Vectors between two points
        vec_p1 = p1-p2
        vec_p2 = p3-p2

        # Calculating the cosine of the angle
        cosine_angle = np.dot(vec_p1,vec_p2) / (np.linalg.norm(vec_p1)*np.linalg.norm(vec_p2))

        # Calculating angle
        angle = np.arccos(cosine_angle)

        # Calculating degrees
        angle_degrees = np.degrees(angle)

        return angle_degrees

    angle_name_list = ['angle_elbow_left',
                   'angle_elbow_right',
                   'angle_shoulder_left',
                   'angle_shoulder_right',
                   'angle_hip_left',
                   'angle_hip_right',
                   'angle_knee_left',
                   'angle_knee_right',
                  ]

    angle_dict = {}
    
    for dictkey, listkey in zip(landmark_dict, angle_name_list):
        i = angle_calculator(landmark_dict[dictkey])
        angle_dict[listkey] = i
    
    angle_list = [angle_dict[key] for key in angle_dict]
    
    return angle_dict, angle_list
    
    
    
    

In [264]:
angle_left_elbow = angle_calculator(landmark_dict['landmarks_left_elbow'])
angle_left_elbow

150.11177

In [265]:
angle_dict = angle_function(lm)[0]

In [266]:
angle_list = angle_function(lm)[1]

In [246]:
# angle_name_list = ['angle_elbow_left',
#                    'angle_elbow_right',
#                    'angle_shoulder_left',
#                    'angle_shoulder_right',
#                    'angle_hip_left',
#                    'angle_hip_right',
#                    'angle_knee_left',
#                    'angle_knee_right',
#                   ]

In [164]:
# angle_dict = {}

In [165]:
# for dictkey, listkey in zip(landmark_dict, angle_name_list):
#     i = angle_calculator(landmark_dict[dictkey])
#     angle_dict[listkey] = i

In [247]:
angle_dict

{'angle_elbow_left': 70.099884,
 'angle_elbow_right': 162.05453,
 'angle_shoulder_left': 35.718674,
 'angle_shoulder_right': 35.58732,
 'angle_hip_left': 69.39518,
 'angle_hip_right': 76.1755,
 'angle_knee_left': 100.24594,
 'angle_knee_right': 134.9033}

These are the angles we need!

In [167]:
# angle_list = [angle_dict[key] for key in angle_dict]

In [170]:
angle_list

[70.099884,
 162.05453,
 35.718674,
 35.58732,
 69.39518,
 76.1755,
 100.24594,
 134.9033]

In [248]:
xyz

[array([[0.61468613, 0.36606914, 0.7225705 ],
        [0.61212206, 0.35098642, 0.6434561 ],
        [0.6129949 , 0.3509376 , 0.7159747 ],
        [0.56388015, 0.3286766 , 0.58742386],
        [0.5627878 , 0.3274661 , 0.5724267 ],
        [0.5053564 , 0.37220737, 0.75928754],
        [0.5080349 , 0.37157354, 0.7369339 ],
        [0.6347408 , 0.26611096, 0.7730443 ],
        [0.6133088 , 0.26804513, 0.77417135],
        [0.70369303, 0.15653855, 0.71411836],
        [0.675892  , 0.1728544 , 0.58133334],
        [0.26079118, 0.6013876 , 0.8471483 ],
        [0.26600903, 0.60989666, 0.8646514 ],
        [0.5165872 , 0.7039327 , 0.8456954 ],
        [0.51504314, 0.70520854, 0.8876169 ],
        [0.6854501 , 0.8830673 , 0.7976013 ],
        [0.667293  , 0.86459327, 0.6523842 ]], dtype=float32),
 array([[0.5346956 , 0.7225141 , 0.56454104],
        [0.5297986 , 0.73396367, 0.40534174],
        [0.5266502 , 0.73250693, 0.39903274],
        [0.4983579 , 0.71041316, 0.46751082],
        [0.491044

In [187]:
lst = []
lst

[]

## Angle Comparison Tool

In [271]:
lst = []
for x in xyz:
    lst.append(angle_function(x)[1])
    
    

In [272]:
lst

[[145.73512,
  105.758606,
  17.348833,
  34.80512,
  65.772285,
  69.11418,
  153.07573,
  123.56796],
 [115.10786,
  147.43588,
  14.766081,
  27.6339,
  69.94517,
  67.07523,
  138.51993,
  121.53896],
 [115.79729,
  172.16551,
  54.05841,
  21.247137,
  89.911705,
  78.78298,
  136.82301,
  144.68764],
 [158.53552,
  48.875996,
  19.07177,
  70.34929,
  73.70838,
  81.390465,
  162.7829,
  114.60971],
 [29.224987,
  157.48364,
  49.513313,
  14.642748,
  65.26562,
  65.18833,
  90.36174,
  169.997],
 [150.11177,
  175.01964,
  6.7305923,
  12.623169,
  65.41411,
  70.32164,
  126.05457,
  139.5287],
 [110.776215,
  158.75452,
  25.847254,
  16.078932,
  75.608894,
  76.07396,
  126.51386,
  87.55917],
 [121.89837,
  68.84903,
  47.64421,
  36.25501,
  78.72896,
  80.194084,
  127.26601,
  177.27911],
 [161.12859,
  172.60306,
  24.238436,
  16.073074,
  51.937447,
  57.386234,
  143.92467,
  135.08281],
 [157.99643,
  155.91643,
  27.688007,
  29.619097,
  85.38014,
  98.84448,
  1

In [273]:
array = np.asarray(lst)

In [274]:
df = pd.DataFrame(lst)

In [282]:
angle_name_list = ['angle_elbow_left',
                   'angle_elbow_right',
                   'angle_shoulder_left',
                   'angle_shoulder_right',
                   'angle_hip_left',
                   'angle_hip_right',
                   'angle_knee_left',
                   'angle_knee_right',
                  ]

In [283]:
df.columns = angle_name_list

In [284]:
df

,angle_elbow_left,angle_elbow_right,angle_shoulder_left,angle_shoulder_right,angle_hip_left,angle_hip_right,angle_knee_left,angle_knee_right
0,145.735123,105.758606,17.348833,34.805119,65.772285,69.114182,153.075729,123.567963
1,115.107857,147.435883,14.766081,27.633900,69.945168,67.075233,138.519928,121.538963
2,115.797287,172.165512,54.058411,21.247137,89.911705,78.782982,136.823013,144.687637
3,158.535522,48.875996,19.071770,70.349289,73.708382,81.390465,162.782898,114.609711
4,29.224987,157.483643,49.513313,14.642748,65.265617,65.188332,90.361740,169.996994
...,...,...,...,...,...,...,...,...
168,126.800255,94.761345,45.961227,50.216599,76.704514,79.578224,148.121292,132.606552
169,133.218170,136.005630,13.299645,40.743114,65.725769,80.648499,109.379379,122.667107
170,88.192329,102.240967,36.759281,39.033100,71.442825,75.393204,130.936142,112.096184
171,121.626282,105.741409,25.741177,51.188892,79.423447,83.175301,160.777420,154.600311


## Angle Error